In [202]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [203]:
df = pd.read_csv("../dataset/facebook_comment_cleaned.tsv",sep='\t',encoding='utf-8')
df['token_arr'] = df['token'].apply(lambda x:eval(x))

In [204]:
from gensim import models
model = models.Word2Vec.load('../model/fbcomment.w2v')

In [205]:
scored = df[df['scoredAmt']>0]

In [206]:
k = 800
def getCommentVector(comment,method='sum'):
    commentVec = np.zeros(800)
    for word in comment:
        try:
            commentVec+=model[word]
        except Exception:
            print(".",end='\r')
    if(method=='avg'):
        commentVec/=len(comment)
    return list(commentVec)
X = scored['token_arr'].apply(getCommentVector)
X = np.array([np.array(i) for i in X])

In [207]:
def giveClass(score):
    if(score==-32 ):
        return -32
    if(score==0):return 0
    if(score>0):return 1
    if(score<0):return -1
scored['class'] = scored['score'].apply(giveClass)
y = scored['class']
Y = pd.get_dummies(y)

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [208]:
X_train,X_test , y_train,y_test = train_test_split(X,Y.values,test_size=0.2,random_state=42)

In [209]:
y_train

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       ..., 
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0]], dtype=uint8)

In [286]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import SGD
model = Sequential()
model.add(Dense(600, activation='relu', input_dim=k))
model.add(Dropout(0.65))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.65))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.55))
model.add(Dense(4, activation='softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.8, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [287]:
model.fit(X_train, y_train,
          epochs=100,
          batch_size=32)

Epoch 1/100
8606/8606 [==============================] - 5s - loss: 6.6045 - acc: 0.4050     
Epoch 2/100
8606/8606 [==============================] - 4s - loss: 4.6777 - acc: 0.4984     
Epoch 3/100
8606/8606 [==============================] - 4s - loss: 2.4238 - acc: 0.5117     
Epoch 4/100
8606/8606 [==============================] - 4s - loss: 1.5726 - acc: 0.5200     
Epoch 5/100
8606/8606 [==============================] - 4s - loss: 1.3437 - acc: 0.5404     
Epoch 6/100
8606/8606 [==============================] - 4s - loss: 1.2101 - acc: 0.5511     
Epoch 7/100
8606/8606 [==============================] - 4s - loss: 1.1599 - acc: 0.5811     
Epoch 8/100
8606/8606 [==============================] - 4s - loss: 1.1919 - acc: 0.5689     
Epoch 9/100
8606/8606 [==============================] - 4s - loss: 1.1583 - acc: 0.5761     
Epoch 10/100
8606/8606 [==============================] - 4s - loss: 1.1069 - acc: 0.5921     - E
Epoch 11/100
8606/8606 [==============================] 

8606/8606 [==============================] - 4s - loss: 1.0354 - acc: 0.6070     
Epoch 87/100
8606/8606 [==============================] - 4s - loss: 1.0036 - acc: 0.6198     
Epoch 88/100
8606/8606 [==============================] - 4s - loss: 0.9998 - acc: 0.6132     
Epoch 89/100
8606/8606 [==============================] - 4s - loss: 1.0000 - acc: 0.6186     
Epoch 90/100
8606/8606 [==============================] - 4s - loss: 1.0340 - acc: 0.6122     
Epoch 91/100
8606/8606 [==============================] - 4s - loss: 1.0176 - acc: 0.6239     
Epoch 92/100
8606/8606 [==============================] - 4s - loss: 1.0189 - acc: 0.6199     
Epoch 93/100
8606/8606 [==============================] - 4s - loss: 1.0231 - acc: 0.6150     
Epoch 94/100
8606/8606 [==============================] - 4s - loss: 1.0109 - acc: 0.6131     
Epoch 95/100
8606/8606 [==============================] - 4s - loss: 1.0168 - acc: 0.6207     
Epoch 96/100
8606/8606 [==============================] - 4s - 

In [288]:
score = model.evaluate(X_test, y_test, batch_size=128)
print("Accuracy: {}%".format(round((score[1]*100),2)))

2152/2152 [==============================] - 1s     
Accuracy: 63.8%


In [151]:
#TODO: use average instead of sum. (But what if length is correlate with ans)
y_pred = np.argmax(model.predict(X_test),axis=1)
y_test_label = np.argmax(y_test,axis=1)

In [152]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test_label)

0.69052044609665431

In [150]:
y_pred
y_test_label

0

FileNotFoundError: File b'../dataset/facebook_comment_annotated_2017.tsv' does not exist